In [1]:
import datetime as dt
import os
import urllib
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import spiceypy as spice
from IPython.display import display, clear_output
from astropy.coordinates import spherical_to_cartesian
from scipy.spatial.distance import cdist
from scipy.spatial.transform import Rotation as R
from surrender.geometry import vec3, quat

import craterdetection.common.constants as const
from craterdetection.common.camera import Camera
from craterdetection.common.conics import generate_mask, crater_representation, ellipse_axes
from craterdetection.common.coordinates import suborbital_coords
from craterdetection.common.spice import get_sol_incidence, get_sun_pos, get_earth_pos
from craterdetection.common.surrender import setup_renderer
from craterdetection.matching.database import extract_robbins_dataset, load_craters

In [2]:
_BASE_URL = 'https://naif.jpl.nasa.gov/pub/naif/'
_KERNEL_ROOT = Path('../data/spice_kernels')
def download_kernel(file_path, base_url=_BASE_URL, base_folder=_KERNEL_ROOT):
    if isinstance(file_path, str):
        file_path = Path(file_path)

    local_path = base_folder / file_path
    url = base_url + file_path.as_posix()

    # Create necessary sub-directories in the DL_PATH direction
    local_path.parent.mkdir(parents=True, exist_ok=True)

    try:
        # If the file is not present in the download directory -> download it
        if not os.path.isfile(local_path):
            print(f"Downloading {url}", end="  ")
            # Download the file with the urllib  package
            urllib.request.urlretrieve(str(url), str(local_path))
            print("Done.")
        else:
            print(f"{base_folder / file_path} already exists!")
    except urllib.error.HTTPError as e:
        print(f"Error: \n{url} could not be found: ", e)

kernels_ = [
               'generic_kernels/lsk/naif0012.tls',
               'generic_kernels/pck/pck00010.tpc',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/spk/de421.bsp',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/pck/moon_pa_de421_1900_2050.bpc',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/moon_assoc_pa.tf',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/moon_080317.tf'
           ]
kernels = list(map(Path, kernels_))

for k in kernels:
    download_kernel(k)

spice.furnsh(list(map(lambda x: str(_KERNEL_ROOT / x), kernels)))
s = setup_renderer()

..\data\spice_kernels\generic_kernels\lsk\naif0012.tls already exists!
..\data\spice_kernels\generic_kernels\pck\pck00010.tpc already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\spk\de421.bsp already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\pck\moon_pa_de421_1900_2050.bpc already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\fk\moon_assoc_pa.tf already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\fk\moon_080317.tf already exists!
Surrender Python Client
code revision : c17439eea9ae041bf49364d67a2a232fc89c1faa


In [3]:
# Constants:
sun_radius = 696342000
earth_radius = 6371e3
ua2km = 149597870.700
ua = ua2km * 1e3
R_moon = 1737.4

s.createBRDF('sun', 'sun.brdf', {})
s.createShape('sun', 'sphere.shp', {'radius': sun_radius})
s.createBody('sun', 'sun', 'sun', [])

s.createBRDF("mate", "mate.brdf", {})
s.createShape("earth_shape", "sphere.shp", {'radius': earth_radius})
s.createBody("earth", "earth_shape", "mate", ["earth.jpg"])

s.createBRDF('hapke', 'hapke.brdf', {})
s.createSphericalDEM('moon', 'FullMoon.dem', 'hapke', 'lroc_color_poles.tiff')
s.setObjectElementBRDF('moon', 'moon', 'hapke')
s.setObjectAttitude('moon', np.array([0, 0, 0, 1]))

s.setObjectPosition('moon', (0, 0, 0))
s.setObjectAttitude('moon', quat(vec3(1, 0, 0), 0))
s.setObjectAttitude('moon', R.from_euler('z', np.pi, degrees=False).as_quat())

In [4]:
lat_cat, long_cat, major_cat, minor_cat, psi_cat, crater_id = extract_robbins_dataset(
    load_craters("../data/lunar_crater_database_robbins_2018.csv", diamlims=[1, 100], ellipse_limit=1.2)
)
r_craters_cat = np.array(np.array(spherical_to_cartesian(const.RMOON, lat_cat, long_cat))).T[..., None]
C_craters_cat = crater_representation(major_cat, minor_cat, psi_cat)

In [5]:
# fig, axes = plt.subplots(1, 2, figsize=(20, 10))

date = dt.date(2020, 12, 2)
et = spice.str2et(str(date))
sun_pos, _ = get_sun_pos(date)
earth_pos, _ = get_earth_pos(date)

s.setObjectPosition('earth', earth_pos*1e3)
s.setObjectPosition('sun', sun_pos*1e3)

cam_alt = 150
axis_threshold = 4

masks_dataset = np.array([])
images_dataset = np.array([])

for i in range(3000):
    sol_incidence = np.inf
    while not sol_incidence <= 85:
        lat = np.random.random()*180 - 90
        long = np.random.random()*360 - 180
        cam = Camera.from_coordinates(lat,
                                      long,
                                      cam_alt,
                                      convert_to_radians=True,
                                      resolution=(256, 256)
                                      )

        r_ilumin = suborbital_coords(cam.r).ravel()
        sol_incidence = get_sol_incidence(date, r_ilumin)

    # Rotations are incremental (order matters)
    cam.rotate('pitch', np.random.random()*40 - 20)
    cam.rotate('yaw', np.random.random()*40 - 20)
    cam.rotate('roll', np.random.random()*40 - 20)

    s.setObjectPosition('camera', cam.r.ravel()*1e3)
    s.setObjectAttitude('camera', R.from_matrix(cam.T).as_quat())

    dist = cam_alt / np.cos(np.radians(cam.fov/2)*np.sqrt(2))
    vicinity = (cdist(r_craters_cat.squeeze(), cam.r.T) < dist*3).ravel()
    r_craters = r_craters_cat[vicinity]
    C_craters = C_craters_cat[vicinity]
    A_craters = cam.project_crater_conics(C_craters, r_craters)

    a_proj, b_proj = ellipse_axes(A_craters)
    axis_filter = np.logical_and(a_proj >= axis_threshold, b_proj >= axis_threshold)
    A_craters = A_craters[axis_filter]
    mask = generate_mask(A_craters, cam.resolution)

    if len(masks_dataset) == 0:
        masks_dataset = mask[None, None, ...]
    else:
        masks_dataset = np.concatenate((masks_dataset, mask[None, None, ...]), axis=0)

    s.render()
    image = s.getImageGray32F()

    if len(images_dataset) == 0:
        images_dataset = image[None, None, ...]
    else:
        images_dataset = np.concatenate((images_dataset, image[None, None, ...]), axis=0)
    print(f"[{i}] Lat: {lat:<5.2f}\t\tLong: {long:.2f}\t\tSolar angle: {sol_incidence:.2f}")

        # axes[0].imshow(image, aspect='equal', interpolation='none', cmap='Greys_r', origin='upper')
        # axes[0].set_axis_off()
        #
        # axes[1].imshow(mask, cmap='Greys_r')
        # axes[1].set_axis_off()
        #
        # clear_output(wait=True)
        # display(fig)
        # print(f"Sun angle: {sol_incidence:.1f} degrees")
        #
        # plt.pause(0.5)

[0] Lat: 9.11 		Long: -106.92		Solar angle: 84.74
[1] Lat: 38.28		Long: 15.02		Solar angle: 51.07
[2] Lat: 49.08		Long: -55.08		Solar angle: 56.36
[3] Lat: -63.37		Long: -53.43		Solar angle: 67.75
[4] Lat: -72.15		Long: -84.68		Solar angle: 82.14
[5] Lat: -7.68		Long: -43.07		Solar angle: 22.28
[6] Lat: 76.30		Long: -34.24		Solar angle: 76.31
[7] Lat: 10.14		Long: -34.83		Solar angle: 15.96
[8] Lat: -57.95		Long: 7.45		Solar angle: 62.82
[9] Lat: -37.61		Long: -41.98		Solar angle: 42.07
[10] Lat: -6.79		Long: -1.56		Solar angle: 21.78
[11] Lat: -30.48		Long: -32.21		Solar angle: 32.21
[12] Lat: 42.54		Long: -67.42		Solar angle: 58.50
[13] Lat: 55.34		Long: -41.12		Solar angle: 57.17
[14] Lat: -54.49		Long: -79.42		Solar angle: 71.92
[15] Lat: 77.38		Long: 40.19		Solar angle: 83.90
[16] Lat: -38.12		Long: 53.30		Solar angle: 78.83
[17] Lat: -70.85		Long: -22.96		Solar angle: 71.15
[18] Lat: -10.11		Long: 5.38		Solar angle: 29.35
[19] Lat: 35.54		Long: -92.84		Solar angle: 74.17
[20] Lat

C:\dev\workspace\crater-detection\craterdetection\common\conics.py:101: RuntimeWarning: invalid value encountered in sqrt
  axes = np.sqrt(1 / lambdas)


In [7]:
with open("output/test_images.npy", "wb") as f:
    np.save(f, images_dataset)

with open("output/test_masks.npy", "wb") as f:
    np.save(f, masks_dataset)